In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
index_name = 'spring-index'
database = PineconeVectorStore.from_existing_index(index_name=index_name)
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")

retriever = database.as_retriever()  # VectorStore에서 retriever 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}  # 필요한 경우 추가 인자 설정
)

dictionary = ["언어를 나타내는 표현 -> 자바"]

prompt = ChatPromptTemplate.from_template(
f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
답변은 한글로 말해주세요.

사전: {dictionary}

질문: {{question}}
""")

chain = prompt | llm | StrOutputParser()
spring_chain = {"query": chain} | qa_chain

In [40]:
new_question = chain.invoke({"question": query})
new_question

'변경할 필요가 없습니다.'

In [42]:
ai_response = spring_chain.invoke({"question": query})
ai_response

{'query': '질문: 자바와 함께 사용되는 프레임워크는 무엇이 있나요',
 'result': '자바와 함께 사용되는 프레임워크에는 Mockito와 JSONassert 같은 테스트 프레임워크가 있으며, FreeMarker, Groovy, Thymeleaf, 그리고 Mustache 같은 템플릿 엔진도 사용됩니다.'}